<a href="https://colab.research.google.com/github/luisscrc/ProyectoFinalIACENTER/blob/main/ProyectoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git ## LIBRERÍA PARA ACELERAR POR GPU
!python rapidsai-csp-utils/colab/pip-install.py ## COMPILAR CON T4

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
Traceback (most recent call last):
  File "/content/rapidsai-csp-utils/colab/pip-install.py", line 28, in <module>
    if ('K80' not in gpu_name):
TypeError: a bytes-like object is required, not 'str'


In [2]:
!wget https://datos.gob.mx/busca/dataset/informacion-referente-a-casos-covid-19-en-mexico ## Comando para descargar algo de una pagina web

--2023-11-13 05:17:24--  https://datos.gob.mx/busca/dataset/informacion-referente-a-casos-covid-19-en-mexico
Resolving datos.gob.mx (datos.gob.mx)... 23.215.7.4, 23.215.7.23
Connecting to datos.gob.mx (datos.gob.mx)|23.215.7.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21597 (21K) [text/html]
Saving to: ‘informacion-referente-a-casos-covid-19-en-mexico.2’

informacion-referen 100%[===================>]  21.09K  --.-KB/s    in 0s      

2023-11-13 05:17:25 (290 MB/s) - ‘informacion-referente-a-casos-covid-19-en-mexico.2’ saved [21597/21597]



In [3]:
!unzip /content/datos_abiertos_covid19.zip

Archive:  /content/datos_abiertos_covid19.zip
replace COVID19MEXICO.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Y
  inflating: COVID19MEXICO.csv       


In [4]:
import pandas as pd
import numpy as np
data=pd.read_csv('COVID19MEXICO.csv',low_memory=False) # RAM INFINITAAAAA
data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'INDIGENA',
       'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'TOMA_MUESTRA_LAB', 'RESULTADO_LAB',
       'TOMA_MUESTRA_ANTIGENO', 'RESULTADO_ANTIGENO', 'CLASIFICACION_FINAL',
       'MIGRANTE', 'PAIS_NACIONALIDAD', 'PAIS_ORIGEN', 'UCI'],
      dtype='object')

In [5]:
df=data

df = df.drop(columns=['SECTOR','MUNICIPIO_RES','HABLA_LENGUA_INDIG','ID_REGISTRO','ORIGEN','PAIS_NACIONALIDAD','PAIS_ORIGEN','INDIGENA','ENTIDAD_UM','ENTIDAD_NAC','NACIONALIDAD','ENTIDAD_RES'])
df.columns

  # FECHA DE ACTUALIZACION; 4 TIPOS DE FECHAS
df['FECHA_ACTUALIZACION']=pd.to_datetime(df['FECHA_ACTUALIZACION'])
df['FECHA_INGRESO']=pd.to_datetime(df['FECHA_INGRESO'])
df['FECHA_SINTOMAS']=pd.to_datetime(df['FECHA_SINTOMAS'])
df['FECHA_DEF']=pd.to_datetime(df['FECHA_DEF'],errors='coerce')

for col in ['TIPO_PACIENTE']:
  df[col]=df[col].astype(str)

from sklearn.preprocessing import LabelEncoder

le= LabelEncoder()


for col in ['TIPO_PACIENTE','SEXO']:
  df[col]=le.fit_transform(df[col])
## Características para darle valor a las fechas
df['DIAS_DEFUNCION']=(df['FECHA_ACTUALIZACION']- df['FECHA_DEF']).dt.days
df['DIAS_HOSPITALIZACION']=(df['FECHA_ACTUALIZACION']- df['FECHA_INGRESO']).dt.days
df['DIAS_SINTOMAS']=(df['FECHA_ACTUALIZACION']- df['FECHA_SINTOMAS']).dt.days
# Red NUERONAL PARA OTORGAR UN 'NaN' y evitar problemas con datos basura
df['DIAS_DEFUNCION'] = df['DIAS_DEFUNCION'].apply(lambda x: 0 if x < 0 else x)

df = df.drop(columns=['FECHA_ACTUALIZACION', 'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF','DIAS_DEFUNCION','MIGRANTE'])

X=df.drop(columns=['CLASIFICACION_FINAL'])
y=df['CLASIFICACION_FINAL']

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [7]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1186694 entries, 0 to 1186693
Data columns (total 25 columns):
 #   Column                 Non-Null Count    Dtype
---  ------                 --------------    -----
 0   SEXO                   1186694 non-null  int64
 1   TIPO_PACIENTE          1186694 non-null  int64
 2   INTUBADO               1186694 non-null  int64
 3   NEUMONIA               1186694 non-null  int64
 4   EDAD                   1186694 non-null  int64
 5   EMBARAZO               1186694 non-null  int64
 6   DIABETES               1186694 non-null  int64
 7   EPOC                   1186694 non-null  int64
 8   ASMA                   1186694 non-null  int64
 9   INMUSUPR               1186694 non-null  int64
 10  HIPERTENSION           1186694 non-null  int64
 11  OTRA_COM               1186694 non-null  int64
 12  CARDIOVASCULAR         1186694 non-null  int64
 13  OBESIDAD               1186694 non-null  int64
 14  RENAL_CRONICA          1186694 non-null  int64
 15

In [8]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
clf=RandomForestClassifier(random_state=42)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.18      0.04      0.07      1673
           2       1.00      0.20      0.33         5
           3       1.00      1.00      1.00     84200
           4       0.08      0.03      0.04        35
           5       0.53      0.53      0.53       396
           6       0.95      0.96      0.96     12816
           7       0.99      1.00      0.99    138214

    accuracy                           0.99    237339
   macro avg       0.68      0.54      0.56    237339
weighted avg       0.98      0.99      0.99    237339



In [9]:
!pip install --upgrade pyarrow

In [10]:
import cuml # PROFE EL CODIGO ESTABA FUNCIONANDO Y TODAS LAS imports AHORA DAN ERRORES
from cuml.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import numpy as np

#
X_train=X_train.astype('float32') ## Acelera para procesamiento por GPU

y_train = y_train.astype('float32')

clf=RandomForestClassifier(random_state=42)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

print(classification_report(y_test,y_pred))

<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.IpcReadOptions size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined

              precision    recall  f1-score   support

           1       1.00      0.01      0.02      1673
           2       0.00      0.00      0.00         5
           3       1.00      1.00      1.00     84200
           4       0.00      0.00      0.00        35
           5       0.59      0.22      0.32       396
           6       0.94      0.97      0.96     12816
           7       0.99      1.00      0.99    138214

    accuracy                           0.99    237339
   macro avg       0.65      0.46      0.47    237339
weighted avg       0.99      0.99      0.99    237339



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
import cuml
from cuml.ensemble import RandomForestClassifier
from cuml.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

cuml_RFC = RandomForestClassifier(random_state=42)
cuml_KNC = KNeighborsClassifier()

classifiers = [ # lista
    {'Random Forest': cuml_RFC},
    {'KNN': cuml_KNC}
]

for classifier in classifiers:
    name, clf = list(classifier.items())[0]  # Extrae el nombre y clasificador del diccionario
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(f'Modelo {name} resultados')
    print(classification_report(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)


Modelo Random Forest resultados


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           1       1.00      0.01      0.02      1673
           2       0.00      0.00      0.00         5
           3       1.00      1.00      1.00     84200
           4       0.00      0.00      0.00        35
           5       0.59      0.22      0.32       396
           6       0.94      0.97      0.96     12816
           7       0.99      1.00      0.99    138214

    accuracy                           0.99    237339
   macro avg       0.65      0.46      0.47    237339
weighted avg       0.99      0.99      0.99    237339

Modelo KNN resultados


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           1       0.35      0.02      0.03      1673
           2       0.00      0.00      0.00         5
           3       0.95      0.84      0.90     84200
           4       0.00      0.00      0.00        35
           5       0.18      0.02      0.03       396
           6       0.96      0.92      0.94     12816
           7       0.90      0.97      0.93    138214

    accuracy                           0.92    237339
   macro avg       0.48      0.40      0.40    237339
weighted avg       0.91      0.92      0.91    237339



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
import locale # configurar compatibilidad de gradio
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [13]:
!pip install gradio # para hacer las paginas web

In [14]:
%load_ext cudf.pandas

In [15]:
import pandas as pd
import numpy as np

def predict(sexo, tipo_paciente, intubado, neumonia, edad, embarazo,
             diabetes, epoc, asma, inmusupr,
            hipertension, otra_com, cardiovascular, obesidad,
            renal_cronica, tabaquismo, otro_caso, toma_muestra_lab,
            resultado_lab, toma_muestra_antigeno, resultado_antigeno,
           uci, dias_hospitalizacion, dias_sintomas):

    # Crea un DataFrame con los valores de entrada
    input_data = pd.DataFrame([[
        sexo, tipo_paciente, intubado, neumonia, edad, embarazo,
       diabetes, epoc, asma, inmusupr,
        hipertension, otra_com, cardiovascular, obesidad,
        renal_cronica, tabaquismo, otro_caso, toma_muestra_lab,
        resultado_lab, toma_muestra_antigeno, resultado_antigeno, uci, dias_hospitalizacion, dias_sintomas
        ]], columns=[
        'SEXO', 'TIPO_PACIENTE', 'INTUBADO', 'NEUMONIA', 'EDAD', 'EMBARAZO','DIABETES', 'EPOC',
        'ASMA', 'INMUSUPR','HIPERTENSION', 'OTRA_COM', 'CARDIOVASCULAR', 'OBESIDAD','RENAL_CRONICA',
        'TABAQUISMO', 'OTRO_CASO', 'TOMA_MUESTRA_LAB','RESULTADO_LAB', 'TOMA_MUESTRA_ANTIGENO',
        'RESULTADO_ANTIGENO','UCI', 'DIAS_HOSPITALIZACION', 'DIAS_SINTOMAS'])

    prediction = clf.predict(input_data)
    return prediction

In [16]:
import gradio as gr # ESTA PARTE NO SÉ PORQUÉ YA NO LA QUISO CORRER PROFE :( ME DA ERROR EN import gradio as gr
from transformers import pipeline

gr.Row
inputs = [ # CONFIGURACION DE LA INTERFAZ GRAFICA
    gr.Radio(choices=[0, 1], label='Sexo'),
    gr.Radio(choices=[0, 1], label='Tipo de Paciente'),
    gr.Radio(choices=[0, 1, 97, 98, 99], label='Intubado'),
    gr.Radio(choices=[0, 1, 97, 98, 99], label='Neumonía'),
    gr.Slider(minimum=0, maximum=120, value=30, label='Edad'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='Embarazo'),
    gr.Radio(choices=[1, 2, 98], label='Diabetes'),
    gr.Radio(choices=[1, 2, 98], label='EPOC'),
    gr.Radio(choices=[1, 2, 98], label='Asma'),
    gr.Radio(choices=[1, 2, 98], label='Inmunosupresión'),
    gr.Radio(choices=[1, 2, 98], label='Hipertensión'),
    gr.Radio(choices=[1, 2, 98], label='Otra Comorbilidad'),
    gr.Radio(choices=[1, 2, 98], label='Cardiovascular'),
    gr.Radio(choices=[1, 2, 98], label='Obesidad'),
    gr.Radio(choices=[1, 2, 98], label='Enfermedad Renal Crónica'),
    gr.Radio(choices=[1, 2, 98], label='Tabaquismo'),
    gr.Radio(choices=[1, 2, 99], label='Contacto con otro caso'),
    gr.Radio(choices=[1, 2, 98], label='Toma de muestra de laboratorio'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='Resultado de laboratorio'),
    gr.Radio(choices=[1, 2, 98], label='Toma de muestra de antígeno'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='Resultado de antígeno'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='UCI'),
    gr.Number(label='Días de Hospitalización',value=0),
    gr.Number(label='Días con Síntomas',value=0),
]

# Crear componente de salida para Gradio
outputs = gr.Textbox(label="Predicción")

# Crear la interfaz de Gradio
demo = gr.Interface(fn=predict, inputs=inputs, outputs=outputs)

# Ejecutar la aplicación web
if __name__ == "__main__":
    demo.launch(show_api=False,debug=True)

ModuleNotFoundError: ignored